In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [ ]:
model_paths = glob.glob('../data/anomalies/*')
models = []
for model_path in model_paths:
    models.append(model_path.split('/')[-1])

In [ ]:
models

In [ ]:
ssps = ['ssp585', 'ssp370', 'ssp245', 'ssp126']

In [ ]:
model_scen_runs = {}
for model_path in model_paths:
    model = model_path.split('/')[-1]
    model_scen_runs[model] = {}
    for ssp in ssps:
        file_paths = glob.glob(f'{model_path}/{ssp}/*')
        model_scen_runs[model][ssp] = []
        for file_path in file_paths:
            file_name = file_path.split('/')[-1]
            run = file_name.split('_')[2]
            model_scen_runs[model][ssp].append(run)
    file_paths = glob.glob(f'{model_path}/historical/*')
    model_scen_runs[model]['historical'] = []
    for file_path in file_paths:
        file_name = file_path.split('/')[-1]
        run = file_name.split('_')[2]
        model_scen_runs[model]['historical'].append(run)

In [ ]:
#model_scen_runs

In [ ]:
base_path = '../data/anomalies'

output_means = {}

for model in model_scen_runs:
    output_means[model] = {}
    for ssp in ssps:
        hist_and_ssp_runs = [run for run in model_scen_runs[model][ssp] if run in model_scen_runs[model]['historical']]
        # discard if the run does not go until at least 2099
        too_short = []
        shortest_run = 10000
        for irun, run in enumerate(hist_and_ssp_runs):
            ssp_df = pd.read_csv(glob.glob(f'{base_path}/{model}/{ssp}/*_{run}_*.csv')[0], index_col='year')
            post_2014_years = len(ssp_df)
            if post_2014_years < 85:
                too_short.append(run)
            else:
                shortest_run = min(shortest_run, post_2014_years+165)
        retained_runs = [run for run in hist_and_ssp_runs if run not in too_short]
        n_runs = len(retained_runs)
        data = np.ones((shortest_run, n_runs)) * np.nan
        for irun, run in enumerate(retained_runs):
            ssp_df = pd.read_csv(glob.glob(f'{base_path}/{model}/{ssp}/*_{run}_*.csv')[0], index_col='year')
            hist_df = pd.read_csv(glob.glob(f'{base_path}/{model}/historical/*_{run}_*.csv')[0], index_col='year')
            full_df = pd.concat((hist_df, ssp_df))
            data[:, irun] = full_df['tas'].values[:shortest_run]
        output_means[model][ssp] = pd.DataFrame(data.mean(axis=1), index=np.arange(1850, 1850+shortest_run), columns=['tas'])

In [ ]:
gwls = [1.5, 2.0, 3.0, 4.0]
crossing_years = []
for model in models:
    for ssp in ssps:
        rolling_df = output_means[model][ssp].rolling(window=19, center=True).mean()['tas']
        row = [model, ssp]
        for gwl in gwls:
            try:
                row.append(rolling_df[rolling_df.gt(gwl)].index[0])
            except IndexError:
                row.append(np.nan)
        crossing_years.append(row)

In [ ]:
df_out = pd.DataFrame(crossing_years, columns=['model', 'scenario', 'GWL1.5', 'GWL2.0', 'GWL3.0', 'GWL4.0'])

In [ ]:
os.makedirs('../output', exist_ok=True)
df_out.to_csv('../output/gwl.csv', index=False)